### 0. 기본환경 세팅

gpu 환경 체크

In [1]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 3050 Ti Laptop GPU (UUID: GPU-b5b4dfdb-dd9f-1ff3-bc8b-d74672661a90)


In [5]:
import torch

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

# output : "cuda"

cpu


In [3]:
# # 데이터셋과 transformers 설치
# %pip install datasets>=2.6.1
# %pip install git+https://github.com/huggingface/transformers

# # 오디오 처리를 위한 librosa 설치
# %pip install librosa

# # 성능 측정을 위한 evaluate와 jiwer 설치
# %pip install evaluate>=0.30
# %pip install jiwer

# # 인터랙티브 인터페이스를 위한 gradio 설치
# %pip install gradio

# # Transformers와 PyTorch를 함께 사용하기 위한 accelerate 설치
# %pip install transformers[torch]
# %pip install accelerate>=0.20.1

In [4]:

from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch

c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. 함수 정의

In [5]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """
    Use Data Collator to perform Speech Seq2Seq with padding
    """
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [6]:
from datasets import load_metric

wer_metric = load_metric("wer")

C:\Users\juneh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}
     

### 2. 음성 처리에 필요한 기능

In [8]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

# Load Feature extractor: WhisperFeatureExtractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

# Load Tokenizer: WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="english", task="transcribe")

# Load Processor: WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="english", task="transcribe")
     

### 3. 데이터셋 불러오기

In [9]:
from datasets import load_from_disk

# Load the combined 'train' dataset
loaded_train_dataset = load_from_disk('./data/map_dataset/train')

# Load the combined 'test' dataset
loaded_test_dataset = load_from_disk('./data/map_dataset/test')


In [10]:
print(loaded_train_dataset)
print(loaded_test_dataset)

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 9523
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 502
})


### 4. 학습 및 평가

In [15]:
# STEP 5.1. Initialize the Data collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

# STEP 5.1. Define evaluation metric
import evaluate
metric = evaluate.load("wer")

# STEP 5.3. Load a pre-trained Checkpoint
from transformers import WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
     

# STEP 5.4. Define the training configuration
"""
Check for Seq2SeqTrainingArguments here:
https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments
"""
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper_base_0401",  # 저장된 모델 및 결과물의 디렉토리 경로
    per_device_train_batch_size=32,  # 한 번에 처리되는 훈련 배치 크기
    gradient_accumulation_steps=2,  # 배치 크기 감소시 그래디언트 누적을 통한 학습 안정화
    learning_rate=1e-5,  # 학습률
    warmup_steps=100,  # 초기 학습률 조정을 위한 웜업 스텝 수 / 일반적으로는 10% ~ 20%의 전체 학습 스텝 수에 해당하는 값을 시도
    max_steps=1000,  # 전체 훈련 스텝 수
    gradient_checkpointing=True,  # 그래디언트 체크포인팅을 통한 메모리 절약
    # fp16=True,  # FP16 형식으로 훈련 수행 (반정밀도 부동소수점)( cpu 가동시 안씀)
    evaluation_strategy="no",  # 검증 수행 전략 설정
    per_device_eval_batch_size=16,  # 한 번에 처리되는 검증 배치 크기
    predict_with_generate=True,  # 생성된 토큰을 통해 예측 수행
    generation_max_length=225,  # 생성된 토큰의 최대 길이 (225유지)
    eval_steps=100,  # 검증 수행 스텝 수
    logging_steps=100,  # 로그 기록 스텝 수
    load_best_model_at_end=False,  # 훈련 종료 시 최적 모델 로드 여부
    metric_for_best_model="wer",  # 최적 모델 선정을 위한 평가 지표 
    greater_is_better=False,  # 평가 지표 값이 높을수록 좋은지 여부
    save_steps=200  # 변경된 save_steps 값
)

# Initialize a trainer.
"""
Forward the training arguments to the Hugging Face trainer along with our model,
dataset, data collator and compute_metrics function.
"""
# 지정된 인자 및 구성요소로 트레이너를 초기화합니다
trainer = Seq2SeqTrainer(
    args=training_args,                   # 이전에 정의한 훈련 인자
    model=model,                          # 훈련할 ASR 모델
    train_dataset=loaded_train_dataset,# 훈련 데이터셋
    eval_dataset=loaded_test_dataset,  # 평가 데이터셋
    data_collator=data_collator,           # 데이터 전처리를 위한 데이터 콜레이터
    compute_metrics=compute_metrics,          # wer 메트릭을 계산하는 함수
    tokenizer=processor.feature_extractor, # 입력 오디오 데이터를 처리하기 위한 토크나이저
)

# Save processor object before starting training
processor.save_pretrained(training_args.output_dir)

# STEP 5.5. Training
"""
Training will take appr. 5-10 hours depending on your GPU.
"""
print('Training 시작')
trainer.train()   # <-- training 시작
print('Training 완료')

#"Step": 모델의 훈련 과정에서 진행되는 각 스텝을 나타내는 숫자입니다.
#스텝은 주로 배치(batch) 단위로 모델이 업데이트되는 지점을 의미합니다.

# Training Loss는 모델이 훈련 데이터에 대해 얼마나 정확하게 예측하는지를 나타내는 지표
# Training Loss가 감소하면 모델이 훈련 데이터에 대해 더 잘 학습하고 있는 것
# 모델이 데이터에 더 잘 적합되고 있다는 것을 의미

# Validation Loss는 모델이 이전에 본 적이 없는 검증 데이터에 대한 예측 정확도
# 훈련 과정 중에 일정 주기마다 검증 데이터를 사용하여 Validation Loss를 계산
# 이 값이 감소하면 모델이 일반화되고 있는 것을 의미
# 모델이 훈련 데이터뿐만 아니라 새로운 데이터에도 잘 예측할 수 있도록 학습되고 있다는 것

# "CER" (Character Error Rate): 훈련 중에 일정 주기마다 검증 데이터를 사용하여
#모델의 문자 에러 비율(CER)을 평가한 값입니다.
#CER은 텍스트 분야에서 자주 사용되는 평가 지표 중 하나로,
#모델이 생성한 텍스트와 실제 텍스트 사이의 문자 수준 오류 비율을 나타냅니다.
#CER이 낮을수록 모델의 성능이 좋다고 판단됩니다.

c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\transformers\optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Training 시작


  6%|▋         | 63/1000 [4:30:01<67:03:19, 257.63s/it]

학습된 모델 평가

In [ ]:
trainer.evaluate()

### 5. 모델 저장

trainer.save_model("./model/whisper_base_0331")

In [ ]:
from transformers import AutoTokenizer

# Load the trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/whisper_base_0824_ver1")

# Specify the directory where you want to save the tokenizer
save_directory = "/content/drive/MyDrive/model/whisper_base_0824"

# Save the tokenizer to the specified directory
tokenizer.save_pretrained(save_directory)

### 6. 모델 실행

In [ ]:
from transformers import pipeline
import soundfile as sf  # soundfile 라이브러리 사용

# ASR 파이프라인 초기화
model_name_or_path = "/content/drive/MyDrive/model/whisper_base_0824"
asr = pipeline(model=model_name_or_path, task="automatic-speech-recognition")

# 음성 파일 경로 리스트
audio_file_paths = [
    "/content/drive/MyDrive/data_file/평가 데이터셋/K00013886-BFG23-L1N2D2-E-K0KK-02601769.wav",
    "/content/drive/MyDrive/data_file/평가 데이터셋/K00013886-BFG23-L1N2D2-E-K0KK-02743434.wav",
    "/content/drive/MyDrive/data_file/평가 데이터셋/K00013886-BFG23-L1N2D2-E-K0KK-02989139.wav",
    "/content/drive/MyDrive/data_file/평가 데이터셋/K00014982-BFG20-L1N2D1-E-K0KK-03006747.wav",
    "/content/drive/MyDrive/data_file/평가 데이터셋/K00014982-BFG20-L1N2D1-E-K0KK-03017978.wav",
    "/content/drive/MyDrive/data_file/평가 데이터셋/K00014982-BFG20-L1N2D4-E-K0KK-02872759.wav",
    # 추가 음성 파일 경로
]

# ASR 함수 정의
def transcribe_audio(audio_path):
    transcription = asr(audio_path)
    return transcription['text']  # Use 'text' key to get the transcribed text

# 각 음성 파일에 대한 처리 및 출력
for audio_file_path in audio_file_paths:
    transcription_text = transcribe_audio(audio_file_path)
    print(f"음성 파일: {audio_file_path}")
    print("텍스트 출력:", transcription_text)
    print()